In [1]:
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline  

import numpy as np
# import tensorflow as tf
from tensorflow.keras.layers import (Activation, Conv3D, Input,Dense, Dropout, Flatten,
                          MaxPooling3D)
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.utils import np_utils
# from tensorflow.keras.utils.vis_utils import plot_model
# from sklearn.model_selection import train_test_split



In [2]:
import pandas as pd
import cv2
import numpy as np
from sklearn.utils import shuffle
import os
from collections import deque
import copy
import matplotlib
import matplotlib.pyplot as plt
# from keras.utils import np_utils

# from config import Config

class ActionDataGenerator(object):
    
    def __init__(self,root_data_path,temporal_stride=1,temporal_length=16,resize=224):
        
        self.root_data_path = root_data_path
        self.temporal_length = temporal_length
        self.temporal_stride = temporal_stride
        self.resize=resize
    def file_generator(self,data_path,data_files):
        '''
        data_files - list of csv files to be read.
        '''
        for f in data_files:       
            tmp_df = pd.read_csv(os.path.join(data_path,f))
            label_list = list(tmp_df['Label'])
            total_images = len(label_list) 
            if total_images>=self.temporal_length:
                num_samples = int((total_images-self.temporal_length)/self.temporal_stride)+1
#                 print ('num of samples from vid seq-{}: {}'.format(f,num_samples))
                img_list = list(tmp_df['FileName'])
            else:
#                 print ('num of frames is less than temporal length; hence discarding this file-{}'.format(f))
                continue
            
            start_frame = 0
            samples = deque()
            samp_count=0
            for img in img_list:
                samples.append(img)
                if len(samples)==self.temporal_length:
                    samples_c=copy.deepcopy(samples)
                    samp_count+=1
                    for t in range(self.temporal_stride):
                        samples.popleft() 
                    yield samples_c,label_list[0]

    def load_samples(self,data_cat='train'):
        data_path = os.path.join(self.root_data_path,data_cat)
        csv_data_files = os.listdir(data_path)
        file_gen = self.file_generator(data_path,csv_data_files)
        iterator = True
        data_list = []
        while iterator:
            try:
                x,y = next(file_gen)
                x=list(x)
                data_list.append([x,y])
            except Exception as e:
#                 print ('the exception: ',e)
                iterator = False
#                 print ('end of data generator')
        return data_list
    
    def shuffle_data(self,samples):
        data = shuffle(samples,random_state=2)
        return data
    
    def preprocess_image(self,img):
        img = cv2.resize(img,(self.resize,self.resize))
        img = img/255
        return img
    
    def data_generator(self,data,batch_size=10,shuffle=True):              
        """
        Yields the next training batch.
        data is an array [[img1_filename,img2_filename...,img16_filename],label1], [image2_filename,label2],...].
        """
        num_samples = len(data)
        if shuffle:
            data = self.shuffle_data(data)
        while True:   
            for offset in range(0, num_samples, batch_size):
                #print ('startring index: ', offset) 
                # Get the samples you'll use in this batch
                batch_samples = data[offset:offset+batch_size]
                # Initialise X_train and y_train arrays for this batch
                X_train = []
                y_train = []
                # For each example
                for batch_sample in batch_samples:
                    # Load image (X)
                    x = batch_sample[0]
                    y = batch_sample[1]
                    temp_data_list = []
                    for img in x:
                        
                        try:
                            img = cv2.imread(img)
                            #apply any kind of preprocessing here
                            #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                            img = self.preprocess_image(img)
                            temp_data_list.append(img)
    
                        except Exception as e:
                            print (e)
#                             print ('error reading file: ',img)  
    
                    # Read label (y)
                    #label = label_names[y]
                    # Add example to arrays
                    X_train.append(temp_data_list)
                    y_train.append(y)
        
                # Make sure they're numpy arrays (as opposed to lists)
                X_train = np.array(X_train)
                #X_train = np.rollaxis(X_train,1,4)
                y_train = np.array(y_train)
                y_train = np.eye(3)[y_train]

                # The generator-y part: yield the next training batch            
                yield X_train, y_train

In [3]:
root_data_path = '/home/ubuntu/ucf_model/activity_data/data_files/'

data_gen_obj=ActionDataGenerator(root_data_path,temporal_stride=1,temporal_length=16,resize=64)



In [4]:
# from tensorflow.keras.models import Sequential
def actmodel(num_classes=3):
    # Define model
    # model = Sequential()
    x_i = Input(shape=(16,64,64,3))
    x = Conv3D(32, kernel_size=(3, 3, 3),  padding='same')(x_i)
    x= Activation('relu')(x)
    x = Conv3D(32, kernel_size=(3, 3, 3), padding='same')(x)
    x=Activation('relu')(x)
    x=MaxPooling3D(pool_size=(3, 3, 3), padding='same')(x)
    x=Dropout(0.25)(x)

    x=Conv3D(64, kernel_size=(3, 3, 3), padding='same')(x)
    x=Activation('relu')(x)
    x=Conv3D(64, kernel_size=(3, 3, 3), padding='same')(x)
    x=Activation('relu')(x)
    x=MaxPooling3D(pool_size=(3, 3, 3), padding='same')(x)
    x=Dropout(0.25)(x)

    x=Flatten()(x)
    x=Dense(512, activation='relu')(x)
    x=Dropout(0.5)(x)
    x=Dense(num_classes)(x)
    y=Activation("softmax")(x)
    # model.compile(loss=categorical_crossentropy,
    #               optimizer=Adam(), metrics=['accuracy'])
    # model.summary()
    #plot_model(model, show_shapes=True,
    #           to_file='model.png')
    model = Model(inputs = x_i,outputs=y)
    model.summary()
    return model

In [5]:
model = actmodel()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 16, 64, 64, 3)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 16, 64, 64, 32)    2624      
_________________________________________________________________
activation (Activation)      (None, 16, 64, 64, 32)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 16, 64, 64, 32)    27680     
_________________________________________________________________
activation_1 (Activation)    (None, 16, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 6, 22, 22, 32)     0         
_________________________________________________________________
dropout (Dropout)            (None, 6, 22, 22, 32)     0     

In [6]:
model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(), metrics=['accuracy'])

In [7]:
train_data = data_gen_obj.load_samples(data_cat='train')

test_data = data_gen_obj.load_samples(data_cat='test')

In [8]:
train_generator = data_gen_obj.data_generator(train_data,batch_size=6,shuffle=True)

test_generator = data_gen_obj.data_generator(test_data,batch_size=6,shuffle=True)

In [10]:
# Fit model using generator
hist = model.fit(train_generator, 
                steps_per_epoch=len(train_data),validation_data=test_generator,validation_steps=len(test_data),epochs=3)

Epoch 1/3
 1118/48976 [..............................] - ETA: 27:29:27 - loss: 1.0875 - accuracy: 0.4000

KeyboardInterrupt: 